Imports

In [ ]:
import librosa
import noisereduce as nr
import numpy as np
import soundfile as sf
from pathlib import Path
import os
import numpy as np
import torch
from torch.utils.data import Dataset
from glob import glob

import os
import numpy as np
import torch


import torch
import torch.nn as nn
import torch.nn.functional as F


import random

from tqdm import tqdm


from tqdm.notebook import tqdm
from torch.utils.data import DataLoader
import torch.optim as optim
import matplotlib.pyplot as plt
import pandas as pd

import shutil


Sample of Vox1celeb dataset

In [ ]:
#Dataset split
base_path = os.path.join(os.getcwd(), 'datasets', 'vox1_dev')
print(base_path)

#Each folder contains a different speaker

speaker_count = sum(os.path.isdir(os.path.join(base_path, entry)) for entry in os.listdir(base_path))
print("Number of original Speakers : ",speaker_count)
#Select a random selection of speakers, as inital dataset of 1211 speakers is roughly 30GB. will take a sample of 150 speakers(12.3%), aroudn 3.7GB


# Destination path for subset
train_path = os.path.join(os.getcwd(), 'datasets', 'vox1_subset_150')
os.makedirs(train_path, exist_ok=True)


# Set seed and select random 150 speakers
random.seed(42)
all_speakers = [entry for entry in os.listdir(base_path)
                if os.path.isdir(os.path.join(base_path, entry))]
selected_speakers = random.sample(all_speakers, 150)

# Copy selected speakers and their full contents to the subset folder
for speaker in selected_speakers:
    src_speaker_path = os.path.join(base_path, speaker)
    dest_speaker_path = os.path.join(train_path, speaker)
    shutil.copytree(src_speaker_path, dest_speaker_path)

#print(f"Subset dataset created at: {train_path}")


# Print the selected speaker folders
count = 0
#print("Selected speakers:")
for speaker in selected_speakers:
    count +=1
    #print(speaker)

print("Number of train Speakers : ",count)



test_path = os.path.join(os.getcwd(), 'datasets', 'vox1_subset_test')

test_speaker_count = sum(os.path.isdir(os.path.join(test_path, entry)) for entry in os.listdir(test_path))
print("Number of test Speakers : ",test_speaker_count)

Splitting that sample into train and val (already a pre split test folder)

In [ ]:

source_path = os.path.join(os.getcwd(), 'datasets', 'vox1_subset_150')

train_folder = os.path.join(source_path, 'train_data')
val_folder = os.path.join(source_path, 'val_data')

os.makedirs(train_folder, exist_ok=True)
os.makedirs(val_folder, exist_ok=True)

speaker_folders = [folder for folder in os.listdir(source_path)
                   if os.path.isdir(os.path.join(source_path, folder)) and folder not in ['train_data', 'val_data']]

random.seed(42)
random.shuffle(speaker_folders)
split_index = int(0.8 * len(speaker_folders))
train_speakers = speaker_folders[:split_index]
val_speakers = speaker_folders[split_index:]

for speaker in train_speakers:
    src = os.path.join(source_path, speaker)
    dst = os.path.join(train_folder, speaker)
    shutil.move(src, dst)

for speaker in val_speakers:
    src = os.path.join(source_path, speaker)
    dst = os.path.join(val_folder, speaker)
    shutil.move(src, dst)

print(f"{len(train_speakers)} speakers moved to train_data, {len(val_speakers)} to val_data.")


Preprocessing and feature extraction


In [ ]:

"""
def preprocess_audio(y, sr, target_duration=6.0, apply_preemphasis=False, apply_reduction=False, coef=0.5, normalise='rms'):
    
    # Trim leading/trailing silence
    y, _ = librosa.effects.trim(y)

    # Apply noise reduction / dereverberation
    if apply_reduction:
        y = nr.reduce_noise(y=y, sr=sr)

    # Apply pre-emphasis filter
    if apply_preemphasis:
        y = librosa.effects.preemphasis(y, coef=coef)

    # Normalisation method
    if normalise == 'rms':
        rms = np.sqrt(np.mean(y**2))
        y = y / (rms + 1e-6)
    elif normalise == 'peak':
        y = y / (np.max(np.abs(y)) + 1e-6)

    # Duration control: pad or truncate
    target_length = int(sr * target_duration)
    if len(y) < target_length:
        y = np.pad(y, (0, target_length - len(y)))
    else:
        y = y[:target_length]

    return y

def preprocess_folder(input_folder, output_folder, sr=22050, target_duration=6.0, apply_preemphasis=False, apply_reduction=False, coef=0.5, normalise='rms'):
    
    input_folder = Path(input_folder)
    output_folder = Path(output_folder)
    wav_files = list(input_folder.rglob("*.wav"))
    
    print(f"Found {len(wav_files)} audio files in {input_folder}")

    for wav_file in wav_files:
        try:
            y, _ = librosa.load(wav_file, sr=sr)
            y = preprocess_audio(
                y, sr, 
                target_duration=target_duration,
                apply_preemphasis=apply_preemphasis,
                apply_reduction=apply_reduction,
                coef=coef,
                normalise=normalise
            )

            # Save processed file to the same structure
            relative_path = wav_file.relative_to(input_folder)
            out_path = output_folder / relative_path
            out_path.parent.mkdir(parents=True, exist_ok=True)

            sf.write(out_path, y, sr)

        except Exception as e:
            print(f"Failed: {wav_file} → {e}")


class AudioFeatureDataset(Dataset):
    def __init__(self, base_path):
        self.base_path = base_path
        self.feature_dirs = {
            "mel": "mel_spectrogram",
            "mfcc": "mfcc",
            "chroma": "chroma",
            "tonnetz": "tonnetz",
            "contrast": "spectral_contrast"
        }

        # Collect relative paths from one feature folder (
        mel_folder = os.path.join(base_path, self.feature_dirs['mel'])
        rel_paths = glob(os.path.join(mel_folder, "*", "*", "*.npy"))
        self.valid_paths = [
            os.path.relpath(path, mel_folder)
            for path in rel_paths
        ]

        # Filter to only keep files that exist in all 5 folders
        self.valid_paths = [
            rel_path for rel_path in self.valid_paths
            if all(
                os.path.exists(os.path.join(base_path, folder, rel_path))
                for folder in self.feature_dirs.values()
            )
        ]

      
        speaker_ids = sorted(set(p.split(os.sep)[0] for p in self.valid_paths))
        self.label_map = {speaker: idx for idx, speaker in enumerate(speaker_ids)}

        print(f"[INFO] Loaded {len(self.valid_paths)} aligned samples")
        print(f"[INFO] Number of speaker classes: {len(self.label_map)}")

    def __len__(self):
        return len(self.valid_paths)

    def __getitem__(self, idx):
        rel_path = self.valid_paths[idx]
        speaker_id = rel_path.split(os.sep)[0]
        label = self.label_map[speaker_id]

        def load(feature):
            path = os.path.join(self.base_path, self.feature_dirs[feature], rel_path)
            return np.load(path)

        mfcc = torch.tensor(load("mfcc"), dtype=torch.float32).mean(dim=1)     
        chroma = torch.tensor(load("chroma"), dtype=torch.float32).mean(dim=1)  
        tonnetz = torch.tensor(load("tonnetz"), dtype=torch.float32).mean(dim=1) 
        contrast = torch.tensor(load("contrast"), dtype=torch.float32).mean(dim=1)  
        mel = torch.tensor(load("mel"), dtype=torch.float32).unsqueeze(0)  


        return mel, mfcc, chroma, tonnetz, contrast, torch.tensor(label)
"""

Cleanier version of pre and featrie exztatoin/ saving

In [ ]:
def preprocess_audio(y, sr, target_duration=6.0, apply_preemphasis=False, apply_reduction=False, coef=0.5, normalise='rms'):
    
    # Trim leading/trailing silence
    y, _ = librosa.effects.trim(y)

    # Apply noise reduction / dereverberation
    if apply_reduction:
        y = nr.reduce_noise(y=y, sr=sr)

    # Apply pre-emphasis filter
    if apply_preemphasis:
        y = librosa.effects.preemphasis(y, coef=coef)

    # Normalisation method
    if normalise == 'rms':
        rms = np.sqrt(np.mean(y**2))
        y = y / (rms + 1e-6)
    elif normalise == 'peak':
        y = y / (np.max(np.abs(y)) + 1e-6)

    # Duration control: pad or truncate
    target_length = int(sr * target_duration)
    if len(y) < target_length:
        y = np.pad(y, (0, target_length - len(y)))
    else:
        y = y[:target_length]

    return y


def extract_and_save_all(input_root, output_root, sr=22050, target_duration=6.0, apply_preemphasis=False, coef=0.5, normalise='rms'):
    input_root = Path(input_root)
    output_root = Path(output_root)

    for split in ["train", "val"]:  
        input_folder = input_root / split
        output_base = output_root / f"preprocessed_{split}"

        print(f"Looking in: {input_folder}")
        wav_files = [f for f in input_folder.glob("*.wav")]
        print(f"Found {len(wav_files)} files in '{split}'")

        for wav_file in tqdm(wav_files):
            try:
                y, _ = librosa.load(wav_file, sr=sr)
                y = preprocess_audio(y, sr, target_duration, apply_preemphasis, coef, normalise)

                base_name = wav_file.stem + ".npy"

                feature_dict = {
                    "mel_spectrogram": librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128),
                    "mfcc": librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20),
                    "chroma": librosa.feature.chroma_stft(y=y, sr=sr),
                    "tonnetz": librosa.feature.tonnetz(y=librosa.effects.harmonic(y), sr=sr),
                    "spectral_contrast": librosa.feature.spectral_contrast(y=y, sr=sr),
                }

                for feature_name, data in feature_dict.items():
                    out_path = output_base / feature_name / base_name
                    out_path.parent.mkdir(parents=True, exist_ok=True)
                    np.save(out_path, data.astype(np.float32))

            except Exception as e:
                print(f"[ERROR] {wav_file.name}: {e}")

#Run the feature extraction

#extract_and_save_all("datasets/release_in_the_wild", "datasets/release_in_the_wild")


#same coide but this tiem test, as its in a different location
def extract_and_save_all(input_root, output_root, sr=22050, target_duration=6.0, apply_preemphasis=False, coef=0.5, normalise='rms'):
    input_root = Path(input_root)
    output_root = Path(output_root)

    for split in ["test"]:  
        input_folder = input_root / split
        output_base = output_root / f"preprocessed_{split}"

        print(f"Looking in: {input_folder}")
        wav_files = [f for f in input_folder.glob("*.wav")]
        print(f"Found {len(wav_files)} files in '{split}'")

        for wav_file in tqdm(wav_files):
            try:
                y, _ = librosa.load(wav_file, sr=sr)
                y = preprocess_audio(y, sr, target_duration, apply_preemphasis, coef, normalise)

                base_name = wav_file.stem + ".npy"

                feature_dict = {
                    "mel_spectrogram": librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128),
                    "mfcc": librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20),
                    "chroma": librosa.feature.chroma_stft(y=y, sr=sr),
                    "tonnetz": librosa.feature.tonnetz(y=librosa.effects.harmonic(y), sr=sr),
                    "spectral_contrast": librosa.feature.spectral_contrast(y=y, sr=sr),
                }

                for feature_name, data in feature_dict.items():
                    out_path = output_base / feature_name / base_name
                    out_path.parent.mkdir(parents=True, exist_ok=True)
                    np.save(out_path, data.astype(np.float32))

            except Exception as e:
                print(f"[ERROR] {wav_file.name}: {e}")

#Run the feature extraction

#extract_and_save_all("datasets/release_in_the_wild", "datasets/release_in_the_wild")

Model Architecture (CNN+LSTM and DenseNN)

In [ ]:



class ConvolutionLSTMBlock(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 32, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.AvgPool2d(2)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.AvgPool2d(2)


        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.AvgPool2d(2)


        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.pool4 = nn.AdaptiveAvgPool2d((32, 8))  

        self.dropout = nn.Dropout(0.1)

        self.lstm = nn.LSTM(input_size=8 * 256, hidden_size=512,
                            num_layers=2, batch_first=True, bidirectional=True)

    def forward(self, x):
        x = F.leaky_relu(self.bn1(self.conv1(x)))
        
        x = self.pool1(x)

        x = F.leaky_relu(self.bn2(self.conv2(x)))
        x = self.pool2(x)


        x = F.leaky_relu(self.bn3(self.conv3(x)))
        x = self.pool3(x)

        x = F.leaky_relu(self.bn4(self.conv4(x)))
        x = self.pool4(x)

        x = self.dropout(x)
        x = x.permute(0, 2, 1, 3).reshape(x.size(0), 32, -1)

        out, _ = self.lstm(x)
        return out[:, -1, :]  



class DenseNeuralNetwork(nn.Module):
    def __init__(self, input_dim, output_dim=128):
        super(DenseNeuralNetwork, self).__init__()
        
        self.fc1 = nn.Linear(input_dim, 256)
        self.bn1 = nn.BatchNorm1d(256)
        
        self.fc2 = nn.Linear(256, 256)
        self.bn2 = nn.BatchNorm1d(256)
        
        self.fc3 = nn.Linear(256, output_dim)
        self.bn3 = nn.BatchNorm1d(output_dim)

        self.dropout = nn.Dropout(0.3)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        
        x = self.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        
        x = self.relu(self.bn3(self.fc3(x)))
        return x
    


class AudioFusionModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.mel_branch = ConvolutionLSTMBlock()
        self.mfcc_branch = DenseNeuralNetwork(input_dim=40)
        self.chroma_branch = DenseNeuralNetwork(input_dim=12)
        self.tonnetz_branch = DenseNeuralNetwork(input_dim=6)
        self.contrast_branch = DenseNeuralNetwork(input_dim=7)

        self.fc_fusion = nn.Sequential(
            nn.Linear(1024 + 4 * 128, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.ReLU(),  
            nn.Dropout(0.3),
        )

        self.embedding_out = nn.Linear(256, 128)  


    def forward(self, mel, mfcc, chroma, tonnetz, contrast):
        mel_out = self.mel_branch(mel)
        mfcc_out = self.mfcc_branch(mfcc)
        chroma_out = self.chroma_branch(chroma)
        tonnetz_out = self.tonnetz_branch(tonnetz)
        contrast_out = self.contrast_branch(contrast)

        combined = torch.cat([mel_out, mfcc_out, chroma_out, tonnetz_out, contrast_out], dim=1)
        fusion = self.fc_fusion(combined)
        embedding = self.embedding_out(fusion)
        return embedding


Tuned Model Architecture
To test replace the code aboev with this version:



In [ ]:
"""
TUNED MODEL ARCHITECTURE
"""

"""


class ConvolutionLSTMBlock(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 32, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.AvgPool2d(2)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.AvgPool2d(2)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.AvgPool2d(2)

        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.pool4 = nn.AdaptiveAvgPool2d((32, 8))  # Fixed output shape

        self.dropout = nn.Dropout(0.3)  # Increased dropout

        self.lstm = nn.LSTM(
            input_size=8 * 256,  # 8 = width after pooling, 256 = channels
            hidden_size=256,     # reduced hidden size for generalisation
            num_layers=2,
            batch_first=True,
            bidirectional=True
        )

    def forward(self, x):
        x = F.leaky_relu(self.bn1(self.conv1(x)))
        x = self.pool1(x)

        x = F.leaky_relu(self.bn2(self.conv2(x)))
        x = self.pool2(x)

        x = F.leaky_relu(self.bn3(self.conv3(x)))
        x = self.pool3(x)

        x = F.leaky_relu(self.bn4(self.conv4(x)))
        x = self.pool4(x)

        x = self.dropout(x)
        x = x.permute(0, 2, 1, 3).reshape(x.size(0), 32, -1)

        out, _ = self.lstm(x)
        return out[:, -1, :]  # final time step output


class DenseNeuralNetwork(nn.Module):
    def __init__(self, input_dim, output_dim=128):
        super(DenseNeuralNetwork, self).__init__()

        self.fc1 = nn.Linear(input_dim, 256)
        self.bn1 = nn.BatchNorm1d(256)

        self.fc2 = nn.Linear(256, 256)
        self.bn2 = nn.BatchNorm1d(256)

        self.fc3 = nn.Linear(256, output_dim)
        self.dropout = nn.Dropout(0.3)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)

        x = self.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)

        x = self.fc3(x)  # no activation here
        return x


class AudioFusionModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.mel_branch = ConvolutionLSTMBlock()
        self.mfcc_branch = DenseNeuralNetwork(input_dim=40)
        self.chroma_branch = DenseNeuralNetwork(input_dim=12)
        self.tonnetz_branch = DenseNeuralNetwork(input_dim=6)
        self.contrast_branch = DenseNeuralNetwork(input_dim=7)

        # Mel output: 512 (from 2x 256 LSTM), others: 4 * 128
        self.fc_fusion = nn.Sequential(
            nn.Linear(512 + 4 * 128, 384),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(384, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
        )

        self.embedding_out = nn.Linear(256, 128)  # Final speaker embedding

    def forward(self, mel, mfcc, chroma, tonnetz, contrast):
        mel_out = self.mel_branch(mel)
        mfcc_out = self.mfcc_branch(mfcc)
        chroma_out = self.chroma_branch(chroma)
        tonnetz_out = self.tonnetz_branch(tonnetz)
        contrast_out = self.contrast_branch(contrast)

        combined = torch.cat([mel_out, mfcc_out, chroma_out, tonnetz_out, contrast_out], dim=1)
        fusion = self.fc_fusion(combined)
        embedding = self.embedding_out(fusion)  # No activation here
        return embedding

        
"""

Triplet loss dataset creation 

In [ ]:


class TripletAudioDataset(torch.utils.data.Dataset):
    def __init__(self, base_dataset):
        self.base_dataset = base_dataset
        self.label_to_indices = self._build_label_index()


    def _build_label_index(self):
        label_to_indices = {}
        print("[INFO] Building label index...")
        for idx in tqdm(range(len(self.base_dataset)), desc="Indexing labels"):
            _, _, _, _, _, label = self.base_dataset[idx]
            label = label.item()
            if label not in label_to_indices:
                label_to_indices[label] = []
            label_to_indices[label].append(idx)
        return label_to_indices


    def __len__(self):
        return len(self.base_dataset)

    def __getitem__(self, anchor_idx):
        anchor = self.base_dataset[anchor_idx]
        anchor_label = anchor[-1].item()

        positive_idx = anchor_idx
        while positive_idx == anchor_idx:
            positive_idx = random.choice(self.label_to_indices[anchor_label])
        positive = self.base_dataset[positive_idx]

        negative_label = random.choice([label for label in self.label_to_indices.keys() if label != anchor_label])
        negative_idx = random.choice(self.label_to_indices[negative_label])
        negative = self.base_dataset[negative_idx]

        return (
            (anchor[:-1], positive[:-1], negative[:-1])
        )



Training loop (using the triplet loss as the learning loss function )

In [ ]:


def train_triplet(model, triplet_dataset, epochs=20, batch_size=32, lr=1e-4, device='cuda', save_csv_path=None):
    model.to(device)
    dataloader = DataLoader(triplet_dataset, batch_size=batch_size, shuffle=True)

    criterion = nn.TripletMarginLoss(margin=1.0)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

    epoch_losses = []

    for epoch in range(epochs):
        model.train()
        total_loss = 0.0
        loop = tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}")

        for batch in loop:
            anchor, positive, negative = batch

            def move_to_device(x):
                return [t.to(device) for t in x]

            anchor = move_to_device(anchor)
            positive = move_to_device(positive)
            negative = move_to_device(negative)

            emb_anchor = model(*anchor)
            emb_positive = model(*positive)
            emb_negative = model(*negative)

            loss = criterion(emb_anchor, emb_positive, emb_negative)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            loop.set_postfix(loss=loss.item())

        avg_loss = total_loss / len(dataloader)
        epoch_losses.append(avg_loss)

        print(f"[Epoch {epoch+1}] Average Loss: {avg_loss:.4f}")
        scheduler.step()
        for param_group in optimizer.param_groups:
            print(f"Learning rate: {param_group['lr']:.6f}")
    
    torch.save(model.state_dict(), "voice_model.pth")
    print("\nModel saved to 'model.pth'")

    # Plotting loss curve
    plt.figure(figsize=(8, 5))
    plt.plot(range(1, epochs+1), epoch_losses, marker='o')
    plt.xlabel("Epoch")
    plt.ylabel("Triplet Loss")
    plt.title("Triplet Loss Curve")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    if save_csv_path:
        df = pd.DataFrame({
            'epoch': list(range(1, epochs + 1)),
            'avg_loss': epoch_losses
        })
        df.to_csv(save_csv_path, index=False)
        print(f"Loss data saved to: {save_csv_path}")



Running Loop

In [ ]:
base_dataset = AudioFeatureDataset("datasets/vox1_subset_150/val_features")
triplet_dataset = TripletAudioDataset(base_dataset)

model = AudioFusionModel()  
train_triplet(model, triplet_dataset, epochs=100, batch_size=32, lr=0.001, device='cuda')    # Save model

torch.save(model.state_dict(), "voice_model.pth")
print("\nModel saved to 'val_model.pth'")
